In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from cuml.linear_model import LogisticRegression
from cuml.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
import gc



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
train.drop(columns=['Soil_Type7', 'Soil_Type15', 'Soil_Type1'], inplace=True) 
test.drop(columns=['Soil_Type7', 'Soil_Type15', 'Soil_Type1'], inplace=True) 

In [ ]:
train['Cover_Type'].hist()

In [ ]:
np.unique(train['Cover_Type'].values, return_counts=True)

In [ ]:
perm_dict = {1:2, 2:1, 3:3, 4:6, 7:4, 6:5, 5:7,}

In [ ]:
inv_perm = {v: k for k, v in perm_dict.items()}
inv_perm

In [ ]:
train['Cover_Type'].replace(perm_dict, inplace=True)
train['Cover_Type'].hist()

In [ ]:
sample_submission.head()

In [ ]:
train.head()

In [ ]:
np.unique(train.Cover_Type.values, return_counts=True)

In [ ]:
test.head()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
np.sum(train.Cover_Type == 7)

In [ ]:
np.sum(train.Cover_Type == 6)

In [ ]:
train = train[train.Cover_Type !=7]
#train = train[train.Cover_Type !=6]
train.shape

In [ ]:
target = train[['Cover_Type']].values
train.drop(['Cover_Type', 'Id'], axis=1, inplace=True)
test.drop(['Id', ], axis=1, inplace=True)

In [ ]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train = RS.transform(train)
test = RS.transform(test)
del RS
gc.collect()
gc.collect()

In [ ]:
np.unique(target, return_counts=True)

In [ ]:
train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof.shape

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=7, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = train[train_index]
    train_target = target[train_index]

    
    val_features = train[val_index]
    val_target = target[val_index]
    print(np.unique(train_target, return_counts=True))
    print(np.unique(val_target, return_counts=True))

    
    model = LogisticRegression(C=1, solver='qn', max_iter=200, tol=0.000000001)
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred.flatten()
    test_preds += model.predict_proba(test)/n_splits
    print(accuracy_score(val_target,val_pred))
    del model
    gc.collect()
    gc.collect()

In [ ]:
print(accuracy_score(target,train_oof))

In [ ]:
test_preds

In [ ]:
preds = np.argmax(test_preds, axis=1)
preds

In [ ]:
np.unique(preds, return_counts=True)

In [ ]:
preds[np.where(preds == 0)[0]] = 1

In [ ]:
sample_submission['Cover_Type'] = preds
sample_submission['Cover_Type'].replace(inv_perm, inplace=True)
sample_submission.head()

In [ ]:
np.unique(sample_submission['Cover_Type'].values)

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

